-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="25ac8d75-ee97-4c88-8028-94ba991d0dba"/>


# Creating Delta Tables

After extracting data from external data sources, load data into the Lakehouse to ensure that all of the benefits of the Databricks platform can be fully leveraged.

While different organizations may have varying policies for how data is initially loaded into Databricks, we typically recommend that early tables represent a mostly raw version of the data, and that validation and enrichment occur in later stages. This pattern ensures that even if data doesn't match expectations with regards to data types or column names, no data will be dropped, meaning that programmatic or manual intervention can still salvage data in a partially corrupted or invalid state.

This lesson will focus primarily on the pattern used to create most tables, **`CREATE TABLE _ AS SELECT`** (CTAS) statements.

## Learning Objectives
By the end of this lesson, you should be able to:
- Use CTAS statements to create Delta Lake tables
- Create new tables from existing views or tables
- Enrich loaded data with additional metadata
- Declare table schema with generated columns and descriptive comments
- Set advanced options to control data location, quality enforcement, and partitioning

<i18n value="ae119ec7-0185-469d-8986-75c0e3d0a68f"/>


## Run Setup

The setup script will create the data and declare necessary values for the rest of this notebook to execute.

In [0]:
%run ../Includes/Classroom-Setup-04.3

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| removing the working directory "dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02"

Validating the locally installed datasets:
| listing local files...(11 seconds)
| completed (11 seconds total)

Creating & using the schema "munirsheikhcloudseekho_0lj9_da_dewd"...(1 seconds)
Predefined tables in "munirsheikhcloudseekho_0lj9_da_dewd":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-

<i18n value="26c0731b-b738-4034-acdc-0dd2761031e4"/>



## Create Table as Select (CTAS)

**`CREATE TABLE AS SELECT`** statements create and populate Delta tables using data retrieved from an input query.

In [0]:
%sql
CREATE OR REPLACE TABLE sales AS
SELECT * FROM parquet.`${da.paths.datasets}/ecommerce/raw/sales-historical`;

DESCRIBE EXTENDED sales;

col_name,data_type,comment
order_id,bigint,null
email,string,null
transaction_timestamp,bigint,null
total_item_quantity,bigint,null
purchase_revenue_in_usd,double,null
unique_items,bigint,null
items,array>,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,


<i18n value="db0df24b-95f8-45bc-8651-44e4af4537e2"/>

 
CTAS statements automatically infer schema information from query results and do **not** support manual schema declaration. 

This means that CTAS statements are useful for external data ingestion from sources with well-defined schema, such as Parquet files and tables.

CTAS statements also do not support specifying additional file options.

We can see how this would present significant limitations when trying to ingest data from CSV files.

In [0]:
%sql
CREATE OR REPLACE TABLE sales_unparsed AS
SELECT * FROM csv.`${da.paths.datasets}/ecommerce/raw/sales-csv`;

SELECT * FROM sales_unparsed;

_c0
order_id|email|transactions_timestamp|total_item_quantity|purchase_revenue_in_usd|unique_items|items
298592|sandovalaustin@holder.com|1592629288475307|1|850.5|1|[{'coupon': 'NEWBED10'
299024|msmith@monroe.com|1592636869915092|2|1092.6|2|[{'coupon': 'NEWBED10'
300048|robertstimothy@hotmail.com|1592649862529478|1|1075.5|1|[{'coupon': 'NEWBED10'
298711|lovejamie@yahoo.com|1592631406799948|1|850.5|1|[{'coupon': 'NEWBED10'
301760|jennifer7054@gmail.com|1592661071882666|1|940.5|1|[{'coupon': 'NEWBED10'
302809|ywhite@kane.org|1592665563660982|1|1075.5|1|[{'coupon': 'NEWBED10'
309136|karen61@hotmail.com|1592689638083947|1|1795.5|1|[{'coupon': 'NEWBED10'
303941|deborah18@conrad-gallagher.com|1592669885794924|1|850.5|1|[{'coupon': 'NEWBED10'
305920|khanedwin@gmail.com|1592676863608194|1|1075.5|1|[{'coupon': 'NEWBED10'


<i18n value="2d99fed5-4e51-45ea-bcbf-e6da504c5e93"/>


To correctly ingest this data to a Delta Lake table, we'll need to use a reference to the files that allows us to specify options.

In the previous lesson, we showed doing this by registering an external table. Here, we'll slightly evolve this syntax to specify the options to a temporary view, and then use this temp view as the source for a CTAS statement to successfully register the Delta table.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW sales_tmp_vw
  (order_id LONG, email STRING, transactions_timestamp LONG, total_item_quantity INTEGER, purchase_revenue_in_usd DOUBLE, unique_items INTEGER, items STRING)
USING CSV
OPTIONS (
  path = "${da.paths.datasets}/ecommerce/raw/sales-csv",
  header = "true",
  delimiter = "|"
);

CREATE TABLE sales_delta AS
  SELECT * FROM sales_tmp_vw;
  
SELECT * FROM sales_delta

order_id,email,transactions_timestamp,total_item_quantity,purchase_revenue_in_usd,unique_items,items
298592,sandovalaustin@holder.com,1592629288475307,1,850.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_F', 'item_name': 'Standard Full Mattress', 'item_revenue_in_usd': 850.5, 'price_in_usd': 945.0, 'quantity': 1}]"
299024,msmith@monroe.com,1592636869915092,2,1092.6,2,"[{'coupon': 'NEWBED10', 'item_id': 'M_PREM_T', 'item_name': 'Premium Twin Mattress', 'item_revenue_in_usd': 985.5, 'price_in_usd': 1095.0, 'quantity': 1}, {'coupon': 'NEWBED10', 'item_id': 'P_DOWN_S', 'item_name': 'Standard Down Pillow', 'item_revenue_in_usd': 107.10000000000001, 'price_in_usd': 119.0, 'quantity': 1}]"
300048,robertstimothy@hotmail.com,1592649862529478,1,1075.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_K', 'item_name': 'Standard King Mattress', 'item_revenue_in_usd': 1075.5, 'price_in_usd': 1195.0, 'quantity': 1}]"
298711,lovejamie@yahoo.com,1592631406799948,1,850.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_F', 'item_name': 'Standard Full Mattress', 'item_revenue_in_usd': 850.5, 'price_in_usd': 945.0, 'quantity': 1}]"
301760,jennifer7054@gmail.com,1592661071882666,1,940.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_Q', 'item_name': 'Standard Queen Mattress', 'item_revenue_in_usd': 940.5, 'price_in_usd': 1045.0, 'quantity': 1}]"
302809,ywhite@kane.org,1592665563660982,1,1075.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_K', 'item_name': 'Standard King Mattress', 'item_revenue_in_usd': 1075.5, 'price_in_usd': 1195.0, 'quantity': 1}]"
309136,karen61@hotmail.com,1592689638083947,1,1795.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_PREM_K', 'item_name': 'Premium King Mattress', 'item_revenue_in_usd': 1795.5, 'price_in_usd': 1995.0, 'quantity': 1}]"
303941,deborah18@conrad-gallagher.com,1592669885794924,1,850.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_F', 'item_name': 'Standard Full Mattress', 'item_revenue_in_usd': 850.5, 'price_in_usd': 945.0, 'quantity': 1}]"
305920,khanedwin@gmail.com,1592676863608194,1,1075.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_STAN_K', 'item_name': 'Standard King Mattress', 'item_revenue_in_usd': 1075.5, 'price_in_usd': 1195.0, 'quantity': 1}]"
298795,samantha4354@hotmail.com,1592632916516773,1,985.5,1,"[{'coupon': 'NEWBED10', 'item_id': 'M_PREM_T', 'item_name': 'Premium Twin Mattress', 'item_revenue_in_usd': 985.5, 'price_in_usd': 1095.0, 'quantity': 1}]"


<i18n value="3f6875ab-5363-43df-8c36-faeff2933cbe"/>

 
## Filtering and Renaming Columns from Existing Tables

Simple transformations like changing column names or omitting columns from target tables can be easily accomplished during table creation.

The following statement creates a new table containing a subset of columns from the **`sales`** table. 

Here, we'll presume that we're intentionally leaving out information that potentially identifies the user or that provides itemized purchase details. We'll also rename our fields with the assumption that a downstream system has different naming conventions than our source data.

In [0]:
%sql
CREATE OR REPLACE TABLE purchases AS
SELECT order_id AS id, transaction_timestamp, purchase_revenue_in_usd AS price
FROM sales;

SELECT * FROM purchases

id,transaction_timestamp,price
257436,1592193956703494,2190.0
257452,1592201856856023,1195.0
257595,1592213317602596,945.0
257847,1592219850060620,2140.0
275392,1592424836322591,850.5
258151,1592225139292956,1045.0
282615,1592504254634073,940.5
258158,1592225276980125,1795.0
281137,1592496576817530,535.5
258387,1592227891252746,59.0


<i18n value="e2eb9bb3-e6ec-4842-85a9-895007411f02"/>


Note that we could have accomplished this same goal with a view, as shown below.

In [0]:
%sql
CREATE OR REPLACE VIEW purchases_vw AS
SELECT order_id AS id, transaction_timestamp, purchase_revenue_in_usd AS price
FROM sales;

SELECT * FROM purchases_vw

id,transaction_timestamp,price
257436,1592193956703494,2190.0
257452,1592201856856023,1195.0
257595,1592213317602596,945.0
257847,1592219850060620,2140.0
275392,1592424836322591,850.5
258151,1592225139292956,1045.0
282615,1592504254634073,940.5
258158,1592225276980125,1795.0
281137,1592496576817530,535.5
258387,1592227891252746,59.0


<i18n value="5961066a-e322-415f-9ddf-0cc34247f5cf"/>

 
## Declare Schema with Generated Columns

As noted previously, CTAS statements do not support schema declaration. We note above that the timestamp column appears to be some variant of a Unix timestamp, which may not be the most useful for our analysts to derive insights. This is a situation where generated columns would be beneficial.

Generated columns are a special type of column whose values are automatically generated based on a user-specified function over other columns in the Delta table (introduced in DBR 8.3).

The code below demonstrates creating a new table while:
1. Specifying column names and types
1. Adding a <a href="https://docs.databricks.com/delta/delta-batch.html#deltausegeneratedcolumns" target="_blank">generated column</a> to calculate the date
1. Providing a descriptive column comment for the generated column

In [0]:
%sql
CREATE OR REPLACE TABLE purchase_dates (
  id STRING, 
  transaction_timestamp LONG, 
  price STRING,
  date DATE GENERATED ALWAYS AS (
    cast(cast(transaction_timestamp/1e6 AS TIMESTAMP) AS DATE))
    COMMENT "generated based on `transactions_timestamp` column")

<i18n value="13084a7f-10a0-453f-aa53-1f75a7e74dd9"/>

 

Because **`date`** is a generated column, if we write to **`purchase_dates`** without providing values for the **`date`** column, Delta Lake automatically computes them.

**NOTE**: The cell below configures a setting to allow for generating columns when using a Delta Lake **`MERGE`** statement. We'll see more on this syntax later in the course.

In [0]:
%sql
SET spark.databricks.delta.schema.autoMerge.enabled=true; 

MERGE INTO purchase_dates a
USING purchases b
ON a.id = b.id
WHEN NOT MATCHED THEN
  INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
10510,0,0,10510


<i18n value="4d535b90-f71e-49ac-8101-45f328ec0349"/>

 
We can see below that all dates were computed correctly as data was inserted, although neither our source data or insert query specified the values in this field.

As with any Delta Lake source, the query automatically reads the most recent snapshot of the table for any query; you never need to run **`REFRESH TABLE`**.

In [0]:
%sql
SELECT * FROM purchase_dates

id,transaction_timestamp,price,date
257436,1592193956703494,2190.0,2020-06-15
257452,1592201856856023,1195.0,2020-06-15
257595,1592213317602596,945.0,2020-06-15
257847,1592219850060620,2140.0,2020-06-15
275392,1592424836322591,850.5,2020-06-17
258151,1592225139292956,1045.0,2020-06-15
282615,1592504254634073,940.5,2020-06-18
258158,1592225276980125,1795.0,2020-06-15
281137,1592496576817530,535.5,2020-06-18
258387,1592227891252746,59.0,2020-06-15


<i18n value="7035036b-6829-4a4e-bbea-9c9bbdc85776"/>



It's important to note that if a field that would otherwise be generated is included in an insert to a table, this insert will fail if the value provided does not exactly match the value that would be derived by the logic used to define the generated column.

We can see this error by uncommenting and running the cell below:

In [0]:
%sql
-- INSERT INTO purchase_dates VALUES
-- (1, 600000000, 42.0, "2020-06-18")

<i18n value="f8f1d9d6-be28-493e-9736-c6384cbbe944"/>


## Add a Table Constraint

The error message above refers to a **`CHECK constraint`**. Generated columns are a special implementation of check constraints.

Because Delta Lake enforces schema on write, Databricks can support standard SQL constraint management clauses to ensure the quality and integrity of data added to a table.

Databricks currently support two types of constraints:
* <a href="https://docs.databricks.com/delta/delta-constraints.html#not-null-constraint" target="_blank">**`NOT NULL`** constraints</a>
* <a href="https://docs.databricks.com/delta/delta-constraints.html#check-constraint" target="_blank">**`CHECK`** constraints</a>

In both cases, you must ensure that no data violating the constraint is already in the table prior to defining the constraint. Once a constraint has been added to a table, data violating the constraint will result in write failure.

Below, we'll add a **`CHECK`** constraint to the **`date`** column of our table. Note that **`CHECK`** constraints look like standard **`WHERE`** clauses you might use to filter a dataset.

In [0]:
%sql
ALTER TABLE purchase_dates ADD CONSTRAINT valid_date CHECK (date > '2020-01-01');

<i18n value="604861c2-0d45-434e-be65-41c1da7b2bbf"/>


Table constraints are shown in the **`TBLPROPERTIES`** field.

In [0]:
%sql
DESCRIBE EXTENDED purchase_dates

col_name,data_type,comment
id,string,null
transaction_timestamp,bigint,null
price,string,null
date,date,generated based on `transactions_timestamp` column
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,munirsheikhcloudseekho_0lj9_da_dewd,
Table,purchase_dates,
Type,MANAGED,


<i18n value="3de6a201-9b87-4d79-9bb1-4fc2f122d55e"/>


## Enrich Tables with Additional Options and Metadata

So far we've only scratched the surface as far as the options for enriching Delta Lake tables.

Below, we show evolving a CTAS statement to include a number of additional configurations and metadata.

Our **`SELECT`** clause leverages two built-in Spark SQL commands useful for file ingestion:
* **`current_timestamp()`** records the timestamp when the logic is executed
* **`input_file_name()`** records the source data file for each record in the table

We also include logic to create a new date column derived from timestamp data in the source.

The **`CREATE TABLE`** clause contains several options:
* A **`COMMENT`** is added to allow for easier discovery of table contents
* A **`LOCATION`** is specified, which will result in an external (rather than managed) table
* The table is **`PARTITIONED BY`** a date column; this means that the data from each date will exist within its own directory in the target storage location

**NOTE**: Partitioning is shown here primarily to demonstrate syntax and impact. Most Delta Lake tables (especially small-to-medium sized data) will not benefit from partitioning. Because partitioning physically separates data files, this approach can result in a small files problem and prevent file compaction and efficient data skipping. The benefits observed in Hive or HDFS do not translate to Delta Lake, and you should consult with an experienced Delta Lake architect before partitioning tables.

**As a best practice, you should default to non-partitioned tables for most use cases when working with Delta Lake.**

In [0]:
%sql
CREATE OR REPLACE TABLE users_pii
COMMENT "Contains PII"
LOCATION "${da.paths.working_dir}/tmp/users_pii"
PARTITIONED BY (first_touch_date)
AS
  SELECT *, 
    cast(cast(user_first_touch_timestamp/1e6 AS TIMESTAMP) AS DATE) first_touch_date, 
    current_timestamp() updated,
    input_file_name() source_file
  FROM parquet.`${da.paths.datasets}/ecommerce/raw/users-historical/`;
  
SELECT * FROM users_pii;

user_id,user_first_touch_timestamp,email,first_touch_date,updated,source_file
UA000000102357351,1592187804331222,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102357772,1592196585484760,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358075,1592198929755893,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358422,1592200681180797,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358489,1592200952155132,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358495,1592200983001857,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358794,1592202111344629,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102358914,1592202501646714,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102359033,1592202891556793,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet
UA000000102359145,1592203235808647,null,2020-06-15,2022-11-13T18:27:10.613+0000,dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02/ecommerce/raw/users-historical/part-00002-tid-531959640415905750-948b4f2d-2d35-46e3-97eb-e6d85d2bf872-7573-1-c000.snappy.parquet


<i18n value="c1aa43f4-9681-4104-824d-c4ce4bc72914"/>

 
The metadata fields added to the table provide useful information to understand when records were inserted and from where. This can be especially helpful if troubleshooting problems in the source data becomes necessary.

All of the comments and properties for a given table can be reviewed using **`DESCRIBE TABLE EXTENDED`**.

**NOTE**: Delta Lake automatically adds several table properties on table creation.

In [0]:
%sql
DESCRIBE EXTENDED users_pii

col_name,data_type,comment
user_id,string,null
user_first_touch_timestamp,bigint,null
email,string,null
first_touch_date,date,null
updated,timestamp,null
source_file,string,null
# Partition Information,,
# col_name,data_type,comment
first_touch_date,date,null
,,


<i18n value="afad6329-9739-4397-b410-7b49ad9118ce"/>

 
Listing the location used for the table reveals that the unique values in the partition column **`first_touch_date`** are used to create data directories.

In [0]:
%python 
files = dbutils.fs.ls(f"{DA.paths.working_dir}/tmp/users_pii")
display(files)

path,name,size,modificationTime
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/_delta_log/,_delta_log/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-15/,first_touch_date=2020-06-15/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-16/,first_touch_date=2020-06-16/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-17/,first_touch_date=2020-06-17/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-18/,first_touch_date=2020-06-18/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-19/,first_touch_date=2020-06-19/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-20/,first_touch_date=2020-06-20/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-21/,first_touch_date=2020-06-21/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-22/,first_touch_date=2020-06-22/,0,0
dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks/tmp/users_pii/first_touch_date=2020-06-23/,first_touch_date=2020-06-23/,0,0


<i18n value="ac7e881c-3076-4997-8dcc-4daa5c84a226"/>


## Cloning Delta Lake Tables
Delta Lake has two options for efficiently copying Delta Lake tables.

**`DEEP CLONE`** fully copies data and metadata from a source table to a target. This copy occurs incrementally, so executing this command again can sync changes from the source to the target location.

In [0]:
%sql
CREATE OR REPLACE TABLE purchases_clone
DEEP CLONE purchases

source_table_size,source_num_of_files,num_removed_files,num_copied_files,removed_files_size,copied_files_size
142117,4,0,4,0,142117


<i18n value="a42c51a7-ac07-4e67-94ea-ed06b6db126c"/>


Because all the data files must be copied over, this can take quite a while for large datasets.

If you wish to create a copy of a table quickly to test out applying changes without the risk of modifying the current table, **`SHALLOW CLONE`** can be a good option. Shallow clones just copy the Delta transaction logs, meaning that the data doesn't move.

In [0]:
%sql
CREATE OR REPLACE TABLE purchases_shallow_clone
SHALLOW CLONE purchases

source_table_size,source_num_of_files,num_removed_files,num_copied_files,removed_files_size,copied_files_size
142117,4,0,0,0,0


<i18n value="e9b91c6a-2581-4a0b-8f84-2e100f053980"/>


In either case, data modifications applied to the cloned version of the table will be tracked and stored separately from the source. Cloning is a great way to set up tables for testing SQL code while still in development.

<i18n value="64bb6112-b07c-424c-a668-da4cfa29be1f"/>


## Summary

In this notebook, we focused primarily on DDL and syntax for creating Delta Lake tables. In the next notebook, we'll explore options for writing updates to tables.

<i18n value="c78f9f19-f13b-49b4-9ad0-20181528f924"/>

 
Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python 
DA.cleanup()

Resetting the learning environment:
| dropping the schema "munirsheikhcloudseekho_0lj9_da_dewd"...(7 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/munirsheikhcloudseekho@gmail.com/data-engineering-with-databricks"...(2 seconds)

Validating the locally installed datasets:
| listing local files...(9 seconds)
| completed (9 seconds total)



-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>